In [1]:
from projet import *
from PLDAC.train_premodel import * # change file name

## 1. Préparation du Dataset

In [2]:
# 1. Chargement du Dataset Iris

iris = datasets.load_iris()
print([key for key in iris])
print(iris['feature_names'])
print(iris['target_names'])

['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [3]:
# Iris DataFrame
iris_df = pd.DataFrame(iris['data'], columns=iris['feature_names'])

# Ajouter une colonne 'label' au DataFrame
iris_df['label'] = iris['target']

# Transformer les numéros de labels en leurs valeurs textuelles
iris_df['label'] = iris_df['label'].map({i:label for i, label in enumerate(iris['target_names'])})

# Résultat
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


## 2. Fabrication du Dataset de textes

In [4]:
# 2.1. Transformation d'une ligne

print(df_row_to_text(iris_df, 145, subject_name="flower", has_unit=False))
print(df_row_to_text(iris_df, 145, subject_name="flower", has_unit=True))

The flower with 6.7 sepal length (cm), 3.0 sepal width (cm), 5.2 petal length (cm) and 2.3 petal width (cm) is a virginica
The flower with 6.7 cm of sepal length, 3.0 cm of sepal width, 5.2 cm of petal length and 2.3 cm of petal width is a virginica


In [5]:
# 2.2. Conversion des samples en texte

iris_text_list = df_texts_list(iris_df, subject_name="flower", has_unit=True)
for text in iris_text_list[:5] + iris_text_list[-5:] :
    print(text)

The flower with 5.1 cm of sepal length, 3.5 cm of sepal width, 1.4 cm of petal length and 0.2 cm of petal width is a setosa
The flower with 4.9 cm of sepal length, 3.0 cm of sepal width, 1.4 cm of petal length and 0.2 cm of petal width is a setosa
The flower with 4.7 cm of sepal length, 3.2 cm of sepal width, 1.3 cm of petal length and 0.2 cm of petal width is a setosa
The flower with 4.6 cm of sepal length, 3.1 cm of sepal width, 1.5 cm of petal length and 0.2 cm of petal width is a setosa
The flower with 5.0 cm of sepal length, 3.6 cm of sepal width, 1.4 cm of petal length and 0.2 cm of petal width is a setosa
The flower with 6.7 cm of sepal length, 3.0 cm of sepal width, 5.2 cm of petal length and 2.3 cm of petal width is a virginica
The flower with 6.3 cm of sepal length, 2.5 cm of sepal width, 5.0 cm of petal length and 1.9 cm of petal width is a virginica
The flower with 6.5 cm of sepal length, 3.0 cm of sepal width, 5.2 cm of petal length and 2.0 cm of petal width is a virginica

In [6]:
# 2.3. Création d'un nouveau DF

iris_text_df = df_to_df_text(iris_df, subject_name="flower", has_unit=True)
iris_text_df.head(10)

,text,label
0,"The flower with 5.1 cm of sepal length, 3.5 cm...",setosa
1,"The flower with 4.9 cm of sepal length, 3.0 cm...",setosa
2,"The flower with 4.7 cm of sepal length, 3.2 cm...",setosa
3,"The flower with 4.6 cm of sepal length, 3.1 cm...",setosa
4,"The flower with 5.0 cm of sepal length, 3.6 cm...",setosa
5,"The flower with 5.4 cm of sepal length, 3.9 cm...",setosa
6,"The flower with 4.6 cm of sepal length, 3.4 cm...",setosa
7,"The flower with 5.0 cm of sepal length, 3.4 cm...",setosa
8,"The flower with 4.4 cm of sepal length, 2.9 cm...",setosa
9,"The flower with 4.9 cm of sepal length, 3.1 cm...",setosa


## 3. Avec modèles classiques et datasets de base

In [7]:
# 3. Modèle : DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
train_clf(clf, iris_df)

['setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'setosa' 'setosa' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor' 'versicolor' 'v

In [8]:
q = iris_df.iloc[0:1,:-1]
print(q)
clf.predict(q)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2


array(['setosa'], dtype=object)

In [9]:
q = "When we have sepal length (cm)=0.3, sepal width (cm)=0.4, petal length (cm)=0.1, petal width (cm)=0.2, what should be the iris type ?"

print("Question -->", q)
print("Réponse -->", traitement_question(clf, q))

Question --> When we have sepal length (cm)=0.3, sepal width (cm)=0.4, petal length (cm)=0.1, petal width (cm)=0.2, what should be the iris type ?
Réponse --> When we have sepal length (cm)=0.3, sepal width (cm)=0.4, petal length (cm)=0.1, petal width (cm)=0.2, the iris type is setosa


## 4. Avec modèles de langage pré-entraînés (GPT-2)

In [12]:
# 4. Ecriture dans un fichier dataset.txt
# Parce qu'on fait passer un fichier de texte pour le training et non un dataset avec un texte et un label

with open('iris_dataset.txt', 'w') as file :
    for text in iris_text_df.iloc[:,0] :
        file.write(text + '.\n')

In [ ]:
# 5. Hugging Face GPT-2 Training (mettre un temps pour calculer le temps du training)
# 15 min sur le dataset iris

train_premodel(model_name="gpt2", dataset_path="./iris_dataset.txt", save_path="./fine-tuned-gpt2")

In [ ]:
# 6.1. Hugging Face GPT-2 Testing

model, tokenizer = get_model_tokenizer(model_path="./fine-tuned-gpt2", tokenizer_path="gpt2")

In [ ]:
# 6.2. Hugging Face GPT-2 Testing

# ---------------------------------
# First row : The flower with 5.1 cm of sepal length, 3.5 cm of sepal width, 1.4 cm of petal length and 0.2 cm of petal width is a setosa.
# Row 51 : The flower with 7.0 cm of sepal length, 3.2 cm of sepal width, 4.7 cm of petal length and 1.4 cm of petal width is a versicolor.
# Row 101 : The flower with 6.3 cm of sepal length, 3.3 cm of sepal width, 6.0 cm of petal length and 2.5 cm of petal width is a virginica.
# ---------------------------------

test_premodel(model, tokenizer, "The flower with")

## 5. A suivre

In [ ]:

# 7. Follow Paper protocols

# 8. Fix errors